In [1]:
%pwd

'd:\\Work\\Chicken-Disease-Classification\\research'

In [2]:
import os

os.chdir("../")

In [3]:
%pwd

'd:\\Work\\Chicken-Disease-Classification'

## Creating entity

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: str

## Create configuration manager

In [5]:
from chicken_disease.constants import *
from chicken_disease.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config

## Create components

In [7]:
import time
import keras

[2023-11-22 15:54:26,249: WARNING: module_wrapper: From d:\Work\Chicken-Disease-Classification\venv\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [8]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

## Create pipeline

In [9]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-11-22 15:54:27,313: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-22 15:54:27,316: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-22 15:54:27,318: INFO: common: created directory at: artifacts]
[2023-11-22 15:54:27,320: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-11-22 15:54:27,322: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
